In [399]:
import stock
from importlib import reload
reload(stock)

import pandas
import time
import os

## Load list of stocks from file

In [400]:
stocks = pandas.read_csv('data/stocks.tcs', index_col=0)
print('Loaded list of %i stocks' % len(stocks))
stocks[-5:]
# stocks = stocks[-2:]

Loaded list of 516 stocks


,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote
Symbol,,,,,,,,
ZION,Zions Bancorporation,24.293,0.000000e+00,NaN,NaN,NaN,NaN,https://www.nasdaq.com/symbol/zb^a
ZNH,China Southern Airlines Company Limited,56.220,1.134314e+10,NaN,1997.0,Transportation,Air Freight/Delivery Services,https://www.nasdaq.com/symbol/znh
ZTS,Zoetis Inc.,89.200,4.316080e+10,NaN,2013.0,Health Care,Major Pharmaceuticals,https://www.nasdaq.com/symbol/zts
V,Visa,NaN,NaN,NaN,NaN,NaN,NaN,https://www.nasdaq.com/symbol/v
NDAQ,Nasdaq,NaN,NaN,NaN,NaN,NaN,NaN,https://www.nasdaq.com/symbol/ndaq


## Download prices history

In [401]:
av = stock.AlphaVantage()
prices = {}
i = 0
total = len(stocks.index)
for symbol in stocks.index:
    i += 1
    try:
        print('[%i/%i] Loading symbol "%s" ...' % (i, total, symbol))
        prices_history = av.monthly_adjusted(symbol=symbol)
        res = 'OK' if stock.ok(prices_history) else (prices_history if type('') == type(prices_history) else 'FAIL')
        print('[%i/%i] Loading symbol "%s" - "%s"' % (i, total, symbol, res))
    except Exception as exc:
        prices_history = None
        print('[%i/%i] Symbol "%s" exception: %r' % (i, total, symbol, exc))
    prices[symbol] = prices_history
    time.sleep(4)

[1/516] Loading symbol "A" ...
[1/516] Loading symbol "A" - "OK"
[2/516] Loading symbol "AA" ...
[2/516] Loading symbol "AA" - "OK"
[3/516] Loading symbol "AABA" ...
[3/516] Loading symbol "AABA" - "OK"
[4/516] Loading symbol "AAL" ...
[4/516] Loading symbol "AAL" - "OK"
[5/516] Loading symbol "AAP" ...
[5/516] Loading symbol "AAP" - "OK"
[6/516] Loading symbol "ABBV" ...
[6/516] Loading symbol "ABBV" - "OK"
[7/516] Loading symbol "ABT" ...
[7/516] Loading symbol "ABT" - "OK"
[8/516] Loading symbol "ACH" ...
[8/516] Loading symbol "ACH" - "OK"
[9/516] Loading symbol "ACN" ...
[9/516] Loading symbol "ACN" - "OK"
[10/516] Loading symbol "ADBE" ...
[10/516] Loading symbol "ADBE" - "OK"
[11/516] Loading symbol "ADI" ...
[11/516] Loading symbol "ADI" - "OK"
[12/516] Loading symbol "ADM" ...
[12/516] Loading symbol "ADM" - "OK"
[13/516] Loading symbol "ADP" ...
[13/516] Loading symbol "ADP" - "OK"
[14/516] Loading symbol "ADS" ...
[14/516] Loading symbol "ADS" - "OK"
[15/516] Loading symbol 

## Try to download failed positions again

In [403]:
i = 0
total = len(prices)
for (symbol, res) in prices.items():    
    i += 1
    if stock.ok(res):
        print('[%i/%i] Symbol "%s" - "OK"' % (i, total, symbol))
        continue
    try:
        print('[%i/%i] Loading symbol "%s" ...' % (i, total, symbol))
        prices_history = av.monthly_adjusted(symbol=symbol)
        res = 'OK' if stock.ok(prices_history) else (prices_history if type('') == type(prices_history) else 'FAIL')
        print('[%i/%i] Loading symbol "%s" - "%s"' % (i, total, symbol, res))
    except Exception as exc:
        prices_history = None
        print('[%i/%i] Symbol "%s" exception: %r' % (i, total, symbol, exc))
    prices[symbol] = prices_history
    time.sleep(4)

[1/513] Symbol "A" - "OK"
[2/513] Symbol "AA" - "OK"
[3/513] Symbol "AABA" - "OK"
[4/513] Symbol "AAL" - "OK"
[5/513] Symbol "AAP" - "OK"
[6/513] Symbol "ABBV" - "OK"
[7/513] Symbol "ABT" - "OK"
[8/513] Symbol "ACH" - "OK"
[9/513] Symbol "ACN" - "OK"
[10/513] Symbol "ADBE" - "OK"
[11/513] Symbol "ADI" - "OK"
[12/513] Symbol "ADM" - "OK"
[13/513] Symbol "ADP" - "OK"
[14/513] Symbol "ADS" - "OK"
[15/513] Symbol "ADSK" - "OK"
[16/513] Symbol "AEE" - "OK"
[17/513] Symbol "AES" - "OK"
[18/513] Symbol "AET" - "OK"
[19/513] Symbol "AFL" - "OK"
[20/513] Symbol "AGN" - "OK"
[21/513] Symbol "AHT" - "OK"
[22/513] Symbol "AIG" - "OK"
[23/513] Symbol "AIV" - "OK"
[24/513] Symbol "AIZ" - "OK"
[25/513] Symbol "AJG" - "OK"
[26/513] Symbol "AKAM" - "OK"
[27/513] Symbol "ALB" - "OK"
[28/513] Symbol "ALGN" - "OK"
[29/513] Symbol "ALK" - "OK"
[30/513] Symbol "ALL" - "OK"
[31/513] Symbol "ALLE" - "OK"
[32/513] Symbol "ALXN" - "OK"
[33/513] Symbol "AMAT" - "OK"
[34/513] Symbol "AME" - "OK"
[35/513] Symbol "

### Remove invalid symbols

In [404]:
prices.pop('LLY.TI', None)
prices.pop('Symbol', None)
prices.pop('VNE$', None)
prices.pop('ZB^A', None)

### List symbols with invalid prices history

In [405]:
invalid_prices = dict([*filter(lambda item: not item[1], prices.items())])
prices = dict([*filter(lambda item: item[1], prices.items())])
print('Invalid prices for symbols: %s' % ', '.join(invalid_prices.keys()))

Invalid prices for symbols: 


## Represet prices history as list of DataFrames

In [406]:
def month_prices_to_dataframe(symbol, prices):
    d = [(symbol, date, price.open, price.close, price.volume, price.high, price.low) for (date, price) in prices.items()]
    return pandas.core.frame.DataFrame(d, columns=['symbol', 'date', 'open', 'close', 'volume', 'high', 'low'])
    

In [407]:
prices_dataframes = []
for (symbol, month_prices) in prices.items():
    df = None
    try:
        df = month_prices_to_dataframe(symbol, month_prices)
        prices_dataframes.append(df)
    except Exception as exc:
        print('DataFrame for symbol "%s" - FAIL' % symbol)
        

DataFrame for symbol "AP3.SG" - FAIL


## Save DataFrames with symbols prices history to "./hostory" directory

In [408]:
def save_dataframe(dir_path, df):
    path = os.path.join(dir_path, df['symbol'][0] + '.csv')
    df.to_csv(path)


In [409]:
history_path = '/Users/fedor/PROJECTS/Stock/history'
for df in prices_dataframes:
    save_dataframe(history_path, df)

### Test saved data

In [410]:
pandas.read_csv(history_path + '/V.csv', index_col=0)

,symbol,date,open,close,volume,high,low
0,V,2018-07-13,131.960000,139.4200,66598734,140.030000,131.150000
1,V,2018-06-29,131.840000,132.4500,154720393,136.690000,129.530000
2,V,2018-05-31,126.860000,130.7200,147562270,132.500000,125.321700
3,V,2018-04-30,119.077577,126.6753,165231144,127.693654,116.521907
4,V,2018-03-29,123.061127,119.4270,182392051,125.237610,115.842792
5,V,2018-02-28,124.538695,122.7416,183890719,126.056242,110.840936
6,V,2018-01-31,114.189022,123.8169,144967375,126.458088,113.571084
7,V,2017-12-29,112.006254,113.6408,175640154,114.537707,106.245477
8,V,2017-11-30,110.132550,112.2156,136052597,113.242175,106.544521
9,V,2017-10-31,105.004623,109.4221,140428442,110.178245,104.365581
